# Store Sales - Time Series Forecasting

Some Ideas
- Use up until 2017 as "train data"

EDA

Models
- Linear Regression, using features of:
store_nbr, family, sales, onpromotion (given - this is more global)
store cluster, store type, store city + state (given - stores csv global)
day of the week, oil price

x days ago sales for (same family, same store, same family + same store)

ignore holidays

### What is the most basic MVP?

EDA
quickly graph:
- aggregated sales per day (across all families)
- sales per day (each family separately)
- aggregated sales per day (each store separately)

- transactions per day (maybe these 2 useless)
- transactions per store

Features and model
- store nbr, family, sales, onpromotion
- x days ago sales (same fam, same store, same fam + store)

- store cluster, store type, store location

# EDA

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time


In [84]:
# Load data
dataPath = './data/'
train = pd.read_csv(dataPath + 'train.csv')
train = train.set_index('id')

stores = pd.read_csv(dataPath + 'stores.csv')
oil = pd.read_csv(dataPath + 'oil.csv')
holidays = pd.read_csv(dataPath + 'holidays_events.csv')

In [85]:
# Combine data into single df
# 1. Store --> match store to (city, state, type, cluster)
# 2. Oil --> match day to oil (fill in na with mean of surrounding days)

# Fill in oil values
oilFilled = oil.bfill()

combined = train.merge(stores, on='store_nbr')
combined = combined.merge(oilFilled, on='date')
combined.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [86]:
# # Graph sales per family
# familySales = combined.groupby(by=['date', 'family'])['sales'].sum().unstack()

# n = len(familySales.columns)
# ncols = 3
# nrows = (n+ncols-1)//ncols
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,nrows*5))

# for i, colName in enumerate(familySales.columns[:n]):
#     currRow = i//ncols
#     currCol = i%ncols

#     familySales[[colName]].plot(rot=30, ax=axs[currRow, currCol])

In [87]:
# # Graph sales per store
# storeSales = combined.groupby(by=['date', 'store_nbr'])['sales'].sum().unstack()

# n = len(storeSales.columns)
# ncols = 3
# nrows = (n+ncols-1)//ncols
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,nrows*5))

# for i, colName in enumerate(storeSales.columns[:n]):
#     currRow = i//ncols
#     currCol = i%ncols

#     storeSales[[colName]].plot(rot=30, ax=axs[currRow, currCol])

# Model

We now want "x days ago" data.

The first such data is "x days ago, what is the mean sales for my given family"
Out of curiosity, let us explore the different methods:

1. Pd.apply. Take in a row --> filter the entire df for the desired date & family --> output mean. (10 sec)
2. Pd.apply. Preprocesses to create a new df with (date, family) : mean. Take in row --> use the preprocessed value. (5 sec)
3. Merge. Preprocess to create a new df with (date, family). Merge df with this preprocessed df. (1 sec)

In [88]:
combined['date'] = pd.to_datetime(combined['date'])
combined['date'].dtype
smallCombined = combined
smallCombined.describe()

,date,store_nbr,sales,onpromotion,cluster,dcoilwtico
count,2143746,2.143746e+06,2.143746e+06,2.143746e+06,2.143746e+06,2.143746e+06
mean,2015-04-24 09:35:45.635909888,2.750000e+01,3.216291e+02,2.831395e+00,8.481481e+00,6.788783e+01
min,2013-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.619000e+01
25%,2014-02-26 00:00:00,1.400000e+01,0.000000e+00,0.000000e+00,4.000000e+00,4.640000e+01
50%,2015-04-24 00:00:00,2.750000e+01,1.000000e+01,0.000000e+00,8.500000e+00,5.330000e+01
75%,2016-06-21 00:00:00,4.100000e+01,1.800000e+02,0.000000e+00,1.300000e+01,9.578000e+01
max,2017-08-15 00:00:00,5.400000e+01,1.247170e+05,7.410000e+02,1.700000e+01,1.106200e+02
std,NaN,1.558579e+01,9.707549e+02,1.324253e+01,4.649735e+00,2.566954e+01


In [ ]:
# Method 1: Create function to filter entire df, for each row
# Runtime:
# Combined 10k length: 5 sec
# Combined 20k length: 17 sec

def prevday_family_mean(row):
    prevDay = row['date'] - pd.Timedelta(days=1)
    mask = (smallCombined['date'] == prevDay) & (smallCombined['family'] == row['family'])
    if not mask.sum():
        return None
    return smallCombined[mask]['sales'].mean()

start_time = time.time()
smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

smallCombined.describe()

Execution time: 5.038007020950317 seconds


/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/450447730.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)


,date,store_nbr,sales,onpromotion,cluster,dcoilwtico,familyMean_1day
count,10000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,6436.000000
mean,2013-01-03 21:38:00.960000,26.948200,169.577629,0.0,8.429100,93.124464,165.549588
min,2013-01-01 00:00:00,1.000000,0.000000,0.0,1.000000,92.970000,0.000000
25%,2013-01-02 00:00:00,14.000000,0.000000,0.0,4.000000,93.120000,0.000000
50%,2013-01-03 00:00:00,27.000000,0.000000,0.0,8.000000,93.140000,0.851852
75%,2013-01-07 00:00:00,39.000000,40.000000,0.0,13.000000,93.200000,67.384481
max,2013-01-08 00:00:00,54.000000,11422.000000,0.0,17.000000,93.210000,3741.111111
std,NaN,15.197513,624.142229,0.0,4.662268,0.078461,502.456439


In [ ]:
# Method 2: Preprocess and then use pd.apply --> get the preprocessed value
# Time:
# 10k length: 5 sec
# 20k length: 16.67 sec
# Slightly faster when not unstacking familySales, and just multiIndexing

def prevday_family_mean_2(row):
    prevDay = row['date'] - pd.Timedelta(day=1)
    return familySales.at[(prevDay, row['family']), :]

start_time = time.time()
familySales = smallCombined.groupby(by=['date', 'family'])['sales'].mean()
smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

smallCombined.describe()
smallCombined.head()

Execution time: 16.683993101119995 seconds


/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/3100960104.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallCombined['familyMean_1day'] = smallCombined.apply(lambda x: prevday_family_mean(x), axis=1)


,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,familyMean_1day
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN


In [95]:
# Method 3: Using merge
# Time:
# Combined length 20k: 0.03 sec
# 

start_time = time.time()
family_sales_shifted = smallCombined.groupby(by=['date', 'family'])['sales'].mean()
family_sales_shifted = family_sales_shifted.unstack().shift(1).stack()
family_sales_shifted.name = 'family_sales_1shift'
small_combined_merged = smallCombined.merge(family_sales_shifted, on=['date', 'family'])

# end_time = time.time()
# print(f"Execution time: {end_time - start_time} seconds")

display(small_combined_merged.head())
display(small_combined_merged.tail())

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,family_sales_1shift
0,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,93.14,0.000000
1,2013-01-02,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,0.000000
2,2013-01-02,1,BEAUTY,2.0,0,Quito,Pichincha,D,13,93.14,0.037037
3,2013-01-02,1,BEVERAGES,1091.0,0,Quito,Pichincha,D,13,93.14,15.000000
4,2013-01-02,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,0.000000


,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,family_sales_1shift
2141959,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,344.398297
2141960,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,86.062500
2141961,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,2134.399926
2141962,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,52.333333
2141963,2017-08-15,9,SEAFOOD,16.000,0,Quito,Pichincha,B,6,47.57,17.975556


In [ ]:
# Bringing it together: doing the day shifts for everything
day_shifts = [1,2,7,8,14,21]

day_family_grouped = smallCombined.groupby(by=['date', 'family'])['sales'].mean().unstack()
for shift in day_shifts:
    day_family_grouped_shifted = day_family_grouped.shift(shift).stack()
    day_family_grouped_shifted.name = f'family_sales_{shift}shift'

    smallCombined = smallCombined.merge(day_family_grouped_shifted, on=['date', 'family'])

day_store_grouped = smallCombined.groupby(by=['date', 'store_nbr'])['sales'].mean().unstack()
for shift in day_shifts:
    day_store_grouped_shifted = day_store_grouped.shift(shift).stack()
    day_store_grouped_shifted.name = f'store_sales_{shift}shift'

    smallCombined = smallCombined.merge(day_store_grouped_shifted, on=['date', 'store_nbr'])

day_family_store_grouped = smallCombined.groupby(by=['date', 'family', 'store_nbr'])['sales'].mean().unstack('store_nbr').unstack('family')
for shift in day_shifts:
    day_family_store_grouped_shifted = day_family_store_grouped.shift(shift).stack('family').stack('store_nbr')
    day_family_store_grouped_shifted.name = f'familystore_sales_{shift}shift'

    smallCombined = smallCombined.merge(day_family_store_grouped_shifted, on=['date', 'family', 'store_nbr'])
display(smallCombined)

/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/1175445068.py:20: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  day_family_store_grouped_shifted = day_family_store_grouped.shift(shift).stack('family').stack('store_nbr')
/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/1175445068.py:20: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  day_family_store_grouped_shifted = day_family_store_grouped.shift(shift).stack('family').stack('store_nbr')
/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_69148/1175445068.py:20: FutureWarning: The 

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,...,familystore_sales_7shift_x,familystore_sales_8shift_x,familystore_sales_14shift_x,familystore_sales_21shift_x,familystore_sales_1shift_y,familystore_sales_2shift_y,familystore_sales_7shift_y,familystore_sales_8shift_y,familystore_sales_14shift_y,familystore_sales_21shift_y
0,2013-07-25,1,AUTOMOTIVE,1.000,0,Quito,Pichincha,D,13,105.47,...,2.000,2.000,1.00000,2.00000,7.000,3.000,2.000,2.000,1.00000,2.00000
1,2013-07-25,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,105.47,...,0.000,0.000,0.00000,0.00000,0.000,0.000,0.000,0.000,0.00000,0.00000
2,2013-07-25,1,BEAUTY,4.000,0,Quito,Pichincha,D,13,105.47,...,5.000,1.000,4.00000,2.00000,3.000,3.000,5.000,1.000,4.00000,2.00000
3,2013-07-25,1,BEVERAGES,911.000,0,Quito,Pichincha,D,13,105.47,...,831.000,819.000,895.00000,874.00000,1066.000,907.000,831.000,819.000,895.00000,874.00000
4,2013-07-25,1,BOOKS,0.000,0,Quito,Pichincha,D,13,105.47,...,0.000,0.000,0.00000,0.00000,0.000,0.000,0.000,0.000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881787,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,...,604.974,425.946,350.09702,434.74802,283.429,525.224,604.974,425.946,350.09702,434.74802
1881788,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,...,117.189,83.426,57.13900,130.92200,114.120,112.100,117.189,83.426,57.13900,130.92200
1881789,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,...,1401.674,1364.578,1059.02300,1599.30100,1348.425,1453.078,1401.674,1364.578,1059.02300,1599.30100
1881790,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,...,136.000,139.000,3.00000,3.00000,182.000,140.000,136.000,139.000,3.00000,3.00000
